In [30]:
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, countDistinct
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [31]:
def init_spark():
    mongo_conn = "mongodb://mongo:27017"
    conf = SparkConf()
    conf.set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")
    conf.set("spark.write.connection.uri", mongo_conn)
    conf.set("spark.mongodb.write.database", "roadtracker")
    conf.set("spark.mongodb.write.collection", "historical1")
    
    sc = SparkContext.getOrCreate(conf=conf)
    
    return SparkSession() \
        .builder \
        .appName("RoadTracker") \
        .getOrCreate()

In [32]:
spark = init_spark()

TypeError: SparkSession.__init__() missing 1 required positional argument: 'sparkContext'

In [39]:
spark = SparkSession \
    .builder \
    .appName("RoadTracker") \
    .getOrCreate()

In [40]:
# load the data
df = spark.read.csv("all_roads.csv", header=True, inferSchema=True)

# read a parquet file
# df = spark.read.parquet("1686522436.8648.parquet", header=True, inferSchema=True)

In [41]:
# historical 1
dfRoadCount = df.groupBy("plate").agg(countDistinct('road')).withColumnRenamed("count(road)", "road_count")

# get the top 100
dfRoadCount = dfRoadCount.orderBy(col("road_count").desc()).limit(100)
dfRoadCount.show()

+-----+----------+
|plate|road_count|
+-----+----------+
|SF428|         1|
|TW315|         1|
|JL736|         1|
|EP171|         1|
|PB094|         1|
|MD954|         1|
|ZZ212|         1|
|ZP226|         1|
|UN926|         1|
|GF380|         1|
|DT049|         1|
|LI295|         1|
|GB081|         1|
|NP659|         1|
|BK572|         1|
|HI030|         1|
|VF070|         1|
|OG955|         1|
|UX361|         1|
|ZP642|         1|
+-----+----------+
only showing top 20 rows



In [42]:
# CALCULATE SPEED AND ACCELERATION

# calculate all speeds achieved by each car
windowDept = Window.partitionBy("plate").orderBy(col("time").desc())
dfCalcs = df.withColumn("row",row_number().over(windowDept))

# calc all speeds
dfCalcs = dfCalcs.withColumn("speed", F.col("x") - F.lag("x", -1).over(windowDept))

# make all values positive
dfCalcs = dfCalcs.withColumn("speed", F.abs(F.col("speed")))

# calc all accs
dfCalcs = dfCalcs.withColumn("acc", F.col("speed") - F.lag("speed", -1).over(windowDept))

# drop nulls and row column
dfCalcs = dfCalcs.na.drop()
dfCalcs = dfCalcs.drop("row")

dfCalcs.show()

+-----+----------+---------+---+---+-----+--------------------+---------+-----+---+
| road|road_speed|road_size|  x|  y|plate|                time|direction|speed|acc|
+-----+----------+---------+---+---+-----+--------------------+---------+-----+---+
|road3|       120|     1000|220|  2|AC272| 1.686422274283066E9|        1|    0|  0|
|road3|       120|     1000|220|  2|AC272|1.6864222536890728E9|        1|    0|  0|
|road3|       120|     1000|220|  2|AC272| 1.686422231388323E9|        1|    0|  0|
|road3|       120|     1000|220|  2|AC272|1.6864222095374138E9|        1|    0|  0|
|road3|       120|     1000|220|  2|AC272|  1.68642220938139E9|        1|    0|  0|
|road3|       120|     1000|220|  2|AC272|1.6864222092032158E9|        1|    0|-35|
|road3|       120|     1000|220|  2|AC272|1.6864222087273128E9|        1|   35|-26|
|road3|       120|     1000|185|  2|AC272|1.6864221860634181E9|        1|   61| -1|
|road3|       120|     1000|331|  9|AU218| 1.686422286230809E9|       -1|   

In [43]:
# historical 2

# get average speed per road
dfStats = dfCalcs.groupBy("road").avg("speed", "road_size")\
            .withColumnRenamed("avg(speed)", "avg_speed")\
            .withColumnRenamed("avg(road_size)", "road_size")

# calculate avg time to cross
dfStats = dfStats.withColumn("avg_time_to_cross", F.col( "road_size") / F.col("avg_speed"))
dfStats = dfStats.select("road", "avg_speed", "avg_time_to_cross")

# get rows where speed = 0 and acc = 0 (collisions)
dfCollisions = dfCalcs.filter((F.col("speed") == 0) & (F.col("acc") == 0))

# group by road and count
dfCollisions = dfCollisions.groupBy("road").count().withColumnRenamed("count", "total_collisions")

# join the dataframes to get all stats
dfStats = dfStats.join(dfCollisions, "road", "left")

In [45]:
# historical 3

# partition by plate and order by time (twice to have ascending and descending row numbers)
windowDept = Window.partitionBy("plate").orderBy(col("time").desc())
windowDept2 = Window.partitionBy("plate").orderBy(col("time").asc())

# create rows columns
dfSpeeds = dfCalcs.withColumn("row",row_number().over(windowDept))
dfSpeeds = dfSpeeds.withColumn("row2",row_number().over(windowDept2))

# check where speed is greater than 120 and the previous speed was less than road_speed (that is, new infraction)
dfSpeeds = dfSpeeds.withColumn("change_in_speed",
                   F.when(((F.col("speed") > F.col("road_speed")) & (F.lag("speed", -1).over(windowDept) <= F.lag("road_speed", -1).over(windowDept) )) , 1) \
                   .otherwise(0))

# check for vehicles that enter a road with speed > road_speed (infraction)
dfSpeeds = dfSpeeds.withColumn("change_in_speed",
                     F.when(((F.col("speed") > F.col("road_speed")) & (F.col("row2") ==1)), 1) \
                        .otherwise(F.col("change_in_speed")))

# chosen T (change it after testing)
t = 2500000000

# get all rows where now() - time < t
dfSpeeds = dfSpeeds.withColumn("past_time", F.unix_timestamp(F.current_timestamp()).cast("double"))
dfSpeeds = dfSpeeds.withColumn("diff_time", F.col("past_time") - F.col("time"))
dfSpeeds = dfSpeeds.filter(F.col("diff_time") < t)

#  check which cars have more than 10 infractions
dfInfractions = dfSpeeds.groupBy("plate").sum("change_in_speed") \
   .withColumnRenamed("sum(change_in_speed)", "total_infractions").filter(F.col("total_infractions") >= 10)

dfInfractions.show()

+-----+-----------------+
|plate|total_infractions|
+-----+-----------------+
+-----+-----------------+



In [ ]:
# analise alternativa
windowDept = Window.partitionBy("plate").orderBy(col("time").desc())
dfCalcs = dfCalcs.withColumn("row",row_number().over(windowDept))

# create column "changed_y" that is 1 if the car changed y and 0 otherwise
dfCalcs = dfCalcs.withColumn("changed_lane",
                     F.when(((F.col("y") != F.lag("y", -1).over(windowDept))), 1) \
                        .otherwise(0))

# create column "over_road_speed" that is 1 if the car is over the road speed and 0 otherwise
dfCalcs = dfCalcs.withColumn("over_road_speed",
                        F.when(((F.col("speed") > F.col("road_speed"))), 1) \
                        .otherwise(0))

# create column "over_acc" that is 1 if (acc>40 or acc<-40) and 0 otherwise
dfCalcs = dfCalcs.withColumn("over_acc",
                        F.when(((F.col("acc") > 40) | (F.col("acc") < -40)), 1) \
                        .otherwise(0))

# create a column "total" that is the sum of the previous columns for each car
# we consider that changing lane 3 times counts as 1 case of unsafe driving
dfCalcs = dfCalcs.withColumn("total", F.col("changed_lane") / 3 + F.col("over_road_speed") + F.col("over_acc"))

# get all rows where now() - time < t (THIS ISNT WHAT WE WANT, TEMPORARY)
#dfCalcs = dfCalcs.withColumn("past_time", F.unix_timestamp(F.current_timestamp()).cast("double"))
#dfCalcs = dfCalcs.withColumn("diff_time", F.col("past_time") - F.col("time"))
#dfCalcs = dfCalcs.filter(F.col("diff_time") < t)

# for each car compute the sum of the column "total" and filter the ones that are greater than 4
dfCalcs = dfCalcs.groupBy("plate").sum("total").withColumnRenamed("sum(total)", "total_infractions").filter(F.col("total_infractions") >= 4)

#dfCalcs.collect()
dfCalcs.select("plate").show()

+-----+
|plate|
+-----+
|VQ482|
+-----+

